<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">Extreme Gradient Boosting with Box Score Features</h4>
    <h5 style="font-weight: bold; font-size: 24px;">Hyperparameter Tuning using Expanding Window</h5>
    <p style="font-size: 20px;">NBA API Seasons 2021-22 to 2023-24</p>
</div>

<a name="Models"></a>

# Table of Contents

[Setup](#Setup)

[Data](#Data)

[Inspect Expanding Training Window](#Inspect-Training-Windows)

[Functions](#Functions)

**[1. Target: Total Points (over / under)](#1.-Target:-Total-Points-(over-/-under))**
  
**[2. Target: Difference in Points (plus / minus)](#2.-Target:-Difference-in-Points-(plus-/-minus))**

**[3. Target: Game Winner (moneyline)](#3.-Target:-Game-Winner-(moneyline))**

# Setup

[Return to top](#Models)

In [1]:
import sys
from pathlib import Path
# get current working directory
cwd = %pwd
# add shared_code directory to Python sys.path
sys.path.append(str(Path(cwd).parent / "shared_code"))
# import all libraries in shared_code directory 'imports.py' file
from imports import *
%matplotlib inline

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


# Data

[Return to top](#Models)

Data splits:

- Define NBA Season 2021-22 as the TRAINING set: regular season is 2021-10-19 to 2022-04-10. 
- Define NBA Season 2022-23 as the VALIDATION set: regular season is 2022-10-18 to 2023-04-09.
- Define NBA Season 2023-24 as the TESTING set: regular season is 2023-10-24 to 2024-04-14.

In [2]:
# load, filter (by time) and scale data
pts_scaled_df, pm_scaled_df, res_scaled_df, test_set_obs = utl.load_and_scale_data(
    file_path='../../data/processed/nba_team_matchups_rolling_box_scores_2022_2024_r05.csv',
    seasons_to_keep=['2021-22', '2022-23', '2023-24'], 
    training_season='2021-22',
    feature_prefix='ROLL_',
    scaler_type='minmax', 
    scale_target=False
)

Season 2021-22: 1186 games
Season 2022-23: 1181 games
Season 2023-24: 692 games
Total number of games across sampled seasons: 3059 games


In [3]:
# define number of games in seasons
season_22_ngames = 1186
season_23_ngames = 1181

In [4]:
pts_scaled_df.head()

,ROLL_HOME_PTS,ROLL_HOME_FGM,ROLL_HOME_FGA,ROLL_HOME_FG_PCT,ROLL_HOME_FG3M,ROLL_HOME_FG3A,ROLL_HOME_FG3_PCT,ROLL_HOME_FTM,ROLL_HOME_FTA,ROLL_HOME_FT_PCT,ROLL_HOME_OREB,ROLL_HOME_DREB,ROLL_HOME_REB,ROLL_HOME_AST,ROLL_HOME_STL,ROLL_HOME_BLK,ROLL_HOME_TOV,ROLL_HOME_PF,ROLL_AWAY_PTS,ROLL_AWAY_FGM,ROLL_AWAY_FGA,ROLL_AWAY_FG_PCT,ROLL_AWAY_FG3M,ROLL_AWAY_FG3A,ROLL_AWAY_FG3_PCT,ROLL_AWAY_FTM,ROLL_AWAY_FTA,ROLL_AWAY_FT_PCT,ROLL_AWAY_OREB,ROLL_AWAY_DREB,ROLL_AWAY_REB,ROLL_AWAY_AST,ROLL_AWAY_STL,ROLL_AWAY_BLK,ROLL_AWAY_TOV,ROLL_AWAY_PF,TOTAL_PTS
GAME_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-23,124.0,42.0,87.0,0.483,16.0,38.0,0.421,24.0,31.0,0.774,13.0,33.0,46.0,26.0,18.0,12.0,18.0,22.0,112.0,42.0,87.0,0.483,15.0,29.0,0.517,13.0,16.0,0.813,9.0,33.0,42.0,26.0,7.0,5.0,16.0,20.0,185
2021-10-23,83.0,30.0,97.0,0.309,7.0,34.0,0.206,16.0,22.0,0.727,19.0,35.0,54.0,14.0,10.0,4.0,19.0,21.0,87.0,31.0,93.0,0.333,13.0,43.0,0.302,12.0,13.0,0.923,10.0,40.0,50.0,16.0,7.0,3.0,15.0,21.0,198
2021-10-23,121.0,45.0,93.0,0.484,12.0,35.0,0.343,19.0,22.0,0.864,9.0,40.0,49.0,25.0,5.0,5.0,12.0,22.0,115.0,42.0,86.0,0.488,10.0,32.0,0.313,21.0,28.0,0.750,7.0,40.0,47.0,31.0,8.0,2.0,11.0,22.0,239
2021-10-23,123.0,49.0,98.0,0.500,13.0,30.0,0.433,12.0,18.0,0.667,13.0,30.0,43.0,32.0,8.0,3.0,8.0,22.0,95.0,32.0,84.0,0.381,12.0,42.0,0.286,19.0,29.0,0.655,5.0,33.0,38.0,19.0,6.0,0.0,15.0,26.0,232
2021-10-24,124.0,48.0,95.0,0.505,17.0,38.0,0.447,11.0,14.0,0.786,10.0,44.0,54.0,29.0,12.0,10.0,17.0,18.0,134.0,48.0,117.0,0.410,21.0,57.0,0.368,17.0,23.0,0.739,15.0,41.0,56.0,34.0,13.0,9.0,18.0,24.0,204


In [5]:
pm_scaled_df.head()

,ROLL_HOME_PTS,ROLL_HOME_FGM,ROLL_HOME_FGA,ROLL_HOME_FG_PCT,ROLL_HOME_FG3M,ROLL_HOME_FG3A,ROLL_HOME_FG3_PCT,ROLL_HOME_FTM,ROLL_HOME_FTA,ROLL_HOME_FT_PCT,ROLL_HOME_OREB,ROLL_HOME_DREB,ROLL_HOME_REB,ROLL_HOME_AST,ROLL_HOME_STL,ROLL_HOME_BLK,ROLL_HOME_TOV,ROLL_HOME_PF,ROLL_AWAY_PTS,ROLL_AWAY_FGM,ROLL_AWAY_FGA,ROLL_AWAY_FG_PCT,ROLL_AWAY_FG3M,ROLL_AWAY_FG3A,ROLL_AWAY_FG3_PCT,ROLL_AWAY_FTM,ROLL_AWAY_FTA,ROLL_AWAY_FT_PCT,ROLL_AWAY_OREB,ROLL_AWAY_DREB,ROLL_AWAY_REB,ROLL_AWAY_AST,ROLL_AWAY_STL,ROLL_AWAY_BLK,ROLL_AWAY_TOV,ROLL_AWAY_PF,PLUS_MINUS
GAME_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-23,124.0,42.0,87.0,0.483,16.0,38.0,0.421,24.0,31.0,0.774,13.0,33.0,46.0,26.0,18.0,12.0,18.0,22.0,112.0,42.0,87.0,0.483,15.0,29.0,0.517,13.0,16.0,0.813,9.0,33.0,42.0,26.0,7.0,5.0,16.0,20.0,7.0
2021-10-23,83.0,30.0,97.0,0.309,7.0,34.0,0.206,16.0,22.0,0.727,19.0,35.0,54.0,14.0,10.0,4.0,19.0,21.0,87.0,31.0,93.0,0.333,13.0,43.0,0.302,12.0,13.0,0.923,10.0,40.0,50.0,16.0,7.0,3.0,15.0,21.0,-8.0
2021-10-23,121.0,45.0,93.0,0.484,12.0,35.0,0.343,19.0,22.0,0.864,9.0,40.0,49.0,25.0,5.0,5.0,12.0,22.0,115.0,42.0,86.0,0.488,10.0,32.0,0.313,21.0,28.0,0.750,7.0,40.0,47.0,31.0,8.0,2.0,11.0,22.0,29.0
2021-10-23,123.0,49.0,98.0,0.500,13.0,30.0,0.433,12.0,18.0,0.667,13.0,30.0,43.0,32.0,8.0,3.0,8.0,22.0,95.0,32.0,84.0,0.381,12.0,42.0,0.286,19.0,29.0,0.655,5.0,33.0,38.0,19.0,6.0,0.0,15.0,26.0,-10.0
2021-10-24,124.0,48.0,95.0,0.505,17.0,38.0,0.447,11.0,14.0,0.786,10.0,44.0,54.0,29.0,12.0,10.0,17.0,18.0,134.0,48.0,117.0,0.410,21.0,57.0,0.368,17.0,23.0,0.739,15.0,41.0,56.0,34.0,13.0,9.0,18.0,24.0,-10.0


In [6]:
res_scaled_df.head()

,ROLL_HOME_PTS,ROLL_HOME_FGM,ROLL_HOME_FGA,ROLL_HOME_FG_PCT,ROLL_HOME_FG3M,ROLL_HOME_FG3A,ROLL_HOME_FG3_PCT,ROLL_HOME_FTM,ROLL_HOME_FTA,ROLL_HOME_FT_PCT,ROLL_HOME_OREB,ROLL_HOME_DREB,ROLL_HOME_REB,ROLL_HOME_AST,ROLL_HOME_STL,ROLL_HOME_BLK,ROLL_HOME_TOV,ROLL_HOME_PF,ROLL_AWAY_PTS,ROLL_AWAY_FGM,ROLL_AWAY_FGA,ROLL_AWAY_FG_PCT,ROLL_AWAY_FG3M,ROLL_AWAY_FG3A,ROLL_AWAY_FG3_PCT,ROLL_AWAY_FTM,ROLL_AWAY_FTA,ROLL_AWAY_FT_PCT,ROLL_AWAY_OREB,ROLL_AWAY_DREB,ROLL_AWAY_REB,ROLL_AWAY_AST,ROLL_AWAY_STL,ROLL_AWAY_BLK,ROLL_AWAY_TOV,ROLL_AWAY_PF,GAME_RESULT
GAME_DATE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-10-23,124.0,42.0,87.0,0.483,16.0,38.0,0.421,24.0,31.0,0.774,13.0,33.0,46.0,26.0,18.0,12.0,18.0,22.0,112.0,42.0,87.0,0.483,15.0,29.0,0.517,13.0,16.0,0.813,9.0,33.0,42.0,26.0,7.0,5.0,16.0,20.0,1
2021-10-23,83.0,30.0,97.0,0.309,7.0,34.0,0.206,16.0,22.0,0.727,19.0,35.0,54.0,14.0,10.0,4.0,19.0,21.0,87.0,31.0,93.0,0.333,13.0,43.0,0.302,12.0,13.0,0.923,10.0,40.0,50.0,16.0,7.0,3.0,15.0,21.0,0
2021-10-23,121.0,45.0,93.0,0.484,12.0,35.0,0.343,19.0,22.0,0.864,9.0,40.0,49.0,25.0,5.0,5.0,12.0,22.0,115.0,42.0,86.0,0.488,10.0,32.0,0.313,21.0,28.0,0.750,7.0,40.0,47.0,31.0,8.0,2.0,11.0,22.0,1
2021-10-23,123.0,49.0,98.0,0.500,13.0,30.0,0.433,12.0,18.0,0.667,13.0,30.0,43.0,32.0,8.0,3.0,8.0,22.0,95.0,32.0,84.0,0.381,12.0,42.0,0.286,19.0,29.0,0.655,5.0,33.0,38.0,19.0,6.0,0.0,15.0,26.0,0
2021-10-24,124.0,48.0,95.0,0.505,17.0,38.0,0.447,11.0,14.0,0.786,10.0,44.0,54.0,29.0,12.0,10.0,17.0,18.0,134.0,48.0,117.0,0.410,21.0,57.0,0.368,17.0,23.0,0.739,15.0,41.0,56.0,34.0,13.0,9.0,18.0,24.0,0


# Inspect Expanding Training Window

[Return to top](#Models)

In [7]:
# expanding window configuration
initial_train_size = 10  # starting size of the training set
test_size = 1            # leave-one-out (LOO) cross-validation
gap_size=0               # should there be a gap between train and test sets?
expansion_limit=None     # the limit on the test set observations

counter = 0
max_splits_to_show = 15

# show first few splits
for train_indices, test_indices in utl.expanding_window_ts_split(pts_scaled_df, initial_train_size, 
                                                                 test_size=test_size, gap_size=gap_size,
                                                                 expansion_limit=expansion_limit):
    print("TRAIN:", train_indices, "TEST:", test_indices)
    counter += 1
    if counter >= max_splits_to_show:
        break

TRAIN: [0 1 2 3 4 5 6 7 8 9] TEST: [10]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10] TEST: [11]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11] TEST: [12]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12] TEST: [13]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13] TEST: [14]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14] TEST: [15]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] TEST: [16]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16] TEST: [17]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17] TEST: [18]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18] TEST: [19]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19] TEST: [20]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20] TEST: [21]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] TEST: [22]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22] TEST: [23]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9

# Functions

[Return to top](#Models)

For some idiotic reason 'XGBoost' requires the training function to be in the notebook rather than imported, at least while using early stopping. Also, we need to pass in validation set data directly to the `.fit` method, which requires a slight modification to the function in the `utl` library.

In [ ]:
def train_with_expanding_window(df, initial_train_size, test_size, target_col, model, gap_size=0,
                                expansion_limit=None, fitted_model=False, ensure_diversity=False):
    """
    Trains a given model using an expanding window approach on a specified DataFrame.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the features and target variable.
    - initial_train_size (int): The initial size of the training dataset.
    - test_size (int): The size of the test dataset for each split, typically 1 for LOO CV.
    - target_col (str): The name of the target column in `df`.
    - model (model object): The instantiated model to be trained, e.g., LinearRegression() or LogisticRegression().
    - gap_size (int): The gap size between training and test datasets. Default is 0.
    - expansion_limit (int, optional): The maximum number of times the training set is expanded by 1 observation during the expanding window process. This parameter controls the total number of train-test splits generated, indirectly determining the final size of the training set. If set, the training process will stop once this limit is reached, potentially leaving some data unused. If None, the training set will expand until all but the last observation are used for training.
    - fitted_model (bool): whether to return the fitted model instance.
    - ensure_diversity (bool, optional): For logistic regression, ensures the initial training data includes both classes. Default is False.

    Returns:
    - model_outputs (list): A list of model predictions or probabilities for the test sets across all splits.
    - y_true (list): A list of the actual target values corresponding to each prediction in `model_outputs`.

    This function iterates over the dataset using an expanding window to create training and test splits, 
    trains the specified `model` on each training split, and stores the model's predictions or probabilities.
    """
    import time
    
    start_time = time.time()

    # initialize storage for model outputs and true labels
    model_outputs = []  # store predictions or probabilities
    y_true = []

    for train_indices, test_indices in utl.expanding_window_ts_split(
        df, initial_train_size, test_size=test_size, gap_size=gap_size,
        expansion_limit=expansion_limit, ensure_diversity=ensure_diversity, 
        target_col=target_col if ensure_diversity else None):
        
        # get training and testing data for this window
        X_train = df.iloc[train_indices].drop(columns=target_col)
        y_train = df.iloc[train_indices][target_col]
        X_test = df.iloc[test_indices].drop(columns=target_col)
        y_test = df.iloc[test_indices][target_col]
        
        # train the model
        model.fit(X_train, y_train)
        
        # check if the model has the predict_proba method (i.e., likely a classifier)
        if hasattr(model, 'predict_proba'):
            # store predicted probabilities of the positive class
            proba = model.predict_proba(X_test)[:, 1]
            model_outputs.extend(list(proba.flatten()))  # ensure it's flattened
        elif hasattr(model, 'predict'):
            # predict for models that support predict (regressors and classifiers without predict_proba)
            predictions = model.predict(X_test)
            model_outputs.extend(list(predictions.flatten()))  # ensure it's flattened
        else:
            raise ValueError("Model does not support required prediction methods.")

        # store true labels for evaluation
        y_true.extend(list(y_test))

    end_time = time.time()
    print(f"Total time taken: {end_time - start_time:.2f} seconds")
    
    if fitted_model:
        return model, model_outputs, y_true
    else:
        return model_outputs, y_true

In [9]:
def train_models_over_grid(df, target_col, initial_train_size, test_size, gap_size,   
                           expansion_limit, model_class, constant_params, explore_params):
    """
    Trains models over a grid of hyperparameters.

    Parameters:
    - df (pd.DataFrame): The dataset to use for training.
    - target_col (str): The name of the target column.
    - initial_train_size (int): Starting size of the training set.
    - test_size (int): Size of the test dataset for each split (LOO cross-validation).
    - gap_size (int): The gap size between training and test datasets. Default is 0.
    - expansion_limit (int): Maximum number of new training observations in expansion.
    - model_class: The class of the model to instantiate.
    - constant_params (dict): Constant parameters for the model.
    - explore_params (dict): Parameters to explore with grid search.

    Returns:
    - dict: A dictionary containing the results for each run.
    """
    import itertools
    import time

    results = {}
    keys, values = zip(*explore_params.items())
    param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    start_time = time.time()

    for i, explore_param in enumerate(param_combinations):
        print('Parameters currently explored:', explore_param)
        
        # instantiate the model with combined parameters
        model = model_class(**constant_params, **explore_param)

        # train over expanding window
        model_outputs, y_true = train_with_expanding_window(
            df=df,
            initial_train_size=initial_train_size,
            test_size=test_size,
            gap_size=gap_size,
            expansion_limit=expansion_limit,
            target_col=target_col,
            model=model
        )
        
        # store outputs and true values in the results dictionary
        results[f"run_{i}"] = {
            "params": {**explore_param},
            "model_outputs": model_outputs,
            "y_true": y_true
        }

    end_time = time.time()
    print(f"Total time taken: {end_time - start_time:.2f} seconds")
    return results

<a name="1.-Target:-Total-Points-(over-/-under)"></a>
# 1. Target: Total Points (over / under)

[Return to top](#Models)

In [9]:
# configuration for expanding window
results = train_models_over_grid(
    model_class=XGBRegressor, # model class
    target_col='TOTAL_PTS', # target column name
    df=pts_scaled_df, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=1,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=50, # maximum number of new training observations in expansion
    constant_params={
         'random_state': 599,
         'n_jobs': -1,
         'objective': 'reg:squarederror',
         'n_estimators': 1000,
         'eval_metric': 'rmse',
         'early_stopping_rounds': 20
    },
    explore_params={
        'booster': ['gbtree'],              # tried: 'gbtree', 'gblinear', 'dart'
        'learning_rate': [0.5, 1.0, 10],    # tried: 0.001, 0.01, 0.1, 0.5, 1.0
        'max_depth': [2, 4, 6],             # tried: 1, 2, 3, 4
        'alpha': [1, 2],                    # tried: 0.1, 1, 2
        'lambda': [1, 5, 10],               # tried: 0.1, 1, 2, 5, 10
        'gamma': [1, 5, 10]                 # tried: 0.1, 1, 2, 5, 10
    }
)

Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 1}
Total time taken: 2.52 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 5}
Total time taken: 2.52 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 10}
Total time taken: 2.53 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 1}
Total time taken: 2.37 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 5}
Total time taken: 2.55 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 10}
Total time taken: 2.64 seconds
Parameters currently explored: {'booster': '

In [10]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_rmse', ascending=True).head()

,run_id,alpha,average_rmse,booster,gamma,lambda,learning_rate,max_depth
94,run_94,1,10.519,gbtree,5,5,1.0,6
73,run_73,1,10.959,gbtree,5,1,1.0,4
74,run_74,1,11.172,gbtree,10,1,1.0,4
92,run_92,1,11.176,gbtree,10,1,1.0,6
90,run_90,1,11.287,gbtree,1,1,1.0,6


In [11]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_rmse')

# save the dictionary to a file
with open('../../hyperparameters/XGB_pts_best_params.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)

<a name="2.-Target:-Difference-in-Points-(plus-/-minus)"></a>
# 2. Target: Difference in Points (plus / minus)

[Return to top](#Models)

In [12]:
# configuration for expanding window
results = train_models_over_grid(
    model_class=XGBRegressor, # model class
    target_col='PLUS_MINUS', # target column name
    df=pm_scaled_df, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=1,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=50, # maximum number of new training observations in expansion
    constant_params={
        'random_state': 599,
        'n_jobs': -1,
        'objective': 'reg:squarederror',
        'n_estimators': 1000,
        'eval_metric': 'rmse',
        'early_stopping_rounds': 20
    },
    explore_params={
        'booster': ['gbtree'],              # tried: 'gbtree', 'gblinear', 'dart'
        'learning_rate': [0.5, 1.0, 10],    # tried: 0.001, 0.01, 0.1, 0.5, 1.0
        'max_depth': [2, 4, 6],             # tried: 1, 2, 3, 4
        'alpha': [1, 2],                    # tried: 0.1, 1, 2
        'lambda': [1, 5, 10],               # tried: 0.1, 1, 2, 5, 10
        'gamma': [1, 5, 10]                 # tried: 0.1, 1, 2, 5, 10
    }
)

Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 1}
Total time taken: 1.95 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 5}
Total time taken: 1.96 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 10}
Total time taken: 1.97 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 1}
Total time taken: 2.36 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 5}
Total time taken: 2.51 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 10}
Total time taken: 2.23 seconds
Parameters currently explored: {'booster': '

In [13]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_rmse', ascending=True).head()

,run_id,alpha,average_rmse,booster,gamma,lambda,learning_rate,max_depth
76,run_76,1,7.216,gbtree,5,5,1.0,4
77,run_77,1,7.541,gbtree,10,5,1.0,4
73,run_73,1,7.667,gbtree,5,1,1.0,4
74,run_74,1,7.685,gbtree,10,1,1.0,4
72,run_72,1,7.767,gbtree,1,1,1.0,4


In [14]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_rmse')

# save the dictionary to a file
with open('../../hyperparameters/XGB_pm_best_params.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)

<a name="3.-Target:-Game-Winner-(moneyline)"></a>
# 3. Target: Game Winner (moneyline)

[Return to top](#Models)

In [15]:
# configuration for expanding window
results = train_models_over_grid(
    model_class=XGBClassifier, # model class
    target_col='GAME_RESULT', # target column name
    df=res_scaled_df, # data set to use
    initial_train_size=season_22_ngames, # starting size of the training set
    test_size=1,  # leave-one-out (LOO) cross-validation
    gap_size = 0,  # should there be a gap between train and test sets?
    expansion_limit=50, # maximum number of new training observations in expansion
    constant_params={
        'random_state': 599,
        'n_jobs': -1,
        'objective': 'binary:logistic',
        'n_estimators': 1000,
        'eval_metric': 'error', # 'auc'
        'early_stopping_rounds': 20
    },
    explore_params={
        'booster': ['gbtree'],              # tried: 'gbtree', 'gblinear', 'dart'
        'learning_rate': [0.5, 1.0, 10],    # tried: 0.001, 0.01, 0.1, 0.5, 1.0
        'max_depth': [2, 4, 6],             # tried: 1, 2, 3, 4
        'alpha': [1, 2],                    # tried: 0.1, 1, 2
        'lambda': [1, 5, 10],               # tried: 0.1, 1, 2, 5, 10
        'gamma': [1, 5, 10]                 # tried: 0.1, 1, 2, 5, 10
    }
)

Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 1}
Total time taken: 1.51 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 5}
Total time taken: 1.42 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 1, 'gamma': 10}
Total time taken: 1.41 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 1}
Total time taken: 1.45 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 5}
Total time taken: 1.44 seconds
Parameters currently explored: {'booster': 'gbtree', 'learning_rate': 0.5, 'max_depth': 2, 'alpha': 1, 'lambda': 5, 'gamma': 10}
Total time taken: 1.40 seconds
Parameters currently explored: {'booster': '

In [16]:
# get metrics for each combination of parameter values
results_df = utl.compile_results_to_dataframe(results)

# print best hyperparameter settings
results_df.sort_values(by='average_accuracy', ascending=False).head()

,run_id,alpha,average_accuracy,average_f1_score,booster,gamma,lambda,learning_rate,max_depth,overall_auc,pred_labels
161,run_161,2,1.0,1.0,gbtree,10,10,10.0,6,1.0,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,..."
143,run_143,2,1.0,1.0,gbtree,10,10,10.0,4,1.0,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,..."
141,run_141,2,1.0,1.0,gbtree,1,10,10.0,4,1.0,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,..."
140,run_140,2,1.0,1.0,gbtree,10,5,10.0,4,1.0,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,..."
147,run_147,1,1.0,1.0,gbtree,1,5,10.0,6,1.0,"[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,..."


In [17]:
# get best parameters from validation as dictionary
best_params = utl.get_best_params(results_df, metric='average_accuracy')

# save the dictionary to a file
with open('../../hyperparameters/XGB_res_best_params.json', 'w') as json_file:
    json.dump(best_params, json_file, default=utl.handle_non_serializable, indent=4)